In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
myLocationData = pd.read_html('https://serwer1990534.home.pl/LocationData.php')

In [7]:
myLocationData[0]

,0,1,2,3
0,Id,Place,Likelihood,Time
1,1,Lubrza,0,2020-04-24 14:54:05
2,2,prudnicki,0,2020-04-24 14:54:06
3,3,48,0,2020-04-24 14:54:06
4,4,opolskie,0,2020-04-24 14:54:06
...,...,...,...,...
96,96,Polska,1,2020-04-24 19:45:45
97,97,Central Europe,1,2020-04-24 19:45:45
98,98,Eastern Europe,1,2020-04-24 19:45:45
99,99,Europa,1,2020-04-24 19:45:46
